# DR8 Calibration Files

The goal of this notebook is to document how we generate the calibration files for DR8 using the burst buffer on Cori.

John Moustakas  
Siena College  
2019 March

### Set up the working directories and burst buffer.

First, choose a new, empty top-level directory on `project` and create it:
```bash
export LEGACY_SURVEY_DIR=/global/project/projectdirs/cosmo/work/legacysurvey/dr8a
mkdir -p $LEGACY_SURVEY_DIR
cd $LEGACY_SURVEY_DIR
```
Note: the same `$LEGACY_SURVEY_DIR` environment variable will need to be defined within the scripts below so that they are self-contained.

**Hereafter I will assume that all scripts are launched from the $LEGACY_SURVEY_DIR directory.**

Next, create soft links to the CP-reduced imaging data.
```bash
mkdir -p images
ln -s /global/project/projectdirs/cosmo/staging/90prime images/90prime
ln -s /global/project/projectdirs/cosmo/staging/mosaic images/mosaic
ln -s /global/project/projectdirs/cosmo/staging/decam images/decam
```

The burst buffer reservation has to be done separately (for DR8 Dustin Lang created a 40TB reservation called "DR8"). You can think of this as a mounted external drive, where large files can be written and read without the significant overhead associated with `$SCRATCH`, although eventually the results will be copied onto `project`.

To access this file system for the first time you have to create a configuration file (just once).
```bash
echo "#DW persistentdw name=DR8" > bb.conf
```

The files in the burst buffer can be accessed *only* from an interactive Cori node.  Let's do this and make a dedicated subdirectory to keep our outputs tidy:
```bash
salloc -N 1 -C haswell -q interactive -t 00:10:00 --bbf=bb.conf
mkdir -p $DW_PERSISTENT_STRIPED_DR8/dr8a
```
Note that the `$DW_PERSISTENT_STRIPED_DR8` environment variable must be used *always*, as every user will have a different absolute path.

### Set up software dependencies.
**ToDo**: Add Docker instructions (link to Adam's NB).  
**ToDo**: Add instructions for compiling tractor and astrometry.net, if needed (separate notebook).  
**ToDo**: QDO may be added to the desiconda stack.

Most of the code dependencies are in the `desiconda` imaging stack (like [tractor](https://github.com/dstndstn/tractor) and [astrometry.net](https://github.com/dstndstn/astrometry.net)), which we will source in our setup script, below, but we usually depend on a recent (hopefully tagged) version of [legacyzpts](https://github.com/legacysurvey/legacyzpts) and [legacypipe](https://github.com/legacysurvey/legacypipe).

```bash
mkdir -p dr8code
git clone git@github.com:legacysurvey/legacyzpts.git dr8code/legacyzpts
git clone git@github.com:legacysurvey/legacypipe.git dr8code/legacypipe
git clone https://bitbucket.org/berkeleylab/qdo/src/master dr8code/qdo
cd legacyzpts ; git checkout tags/dr8 ; cd ..
cd legacypipe ; git checkout tags/dr8 ; cd ..
```

Next, create a bash script called `dr8-env.sh` that will set up all our dependencies and myriad environment variables.  This script isn't checked in because it contains a *qdo* database password, but it should look something like this:

```bash
#! /bin/bash

# Use local check-outs of qdo, legacypipe, and legacyzpts.
export QDO_DIR=$LEGACY_SURVEY_DIR/dr8code/qdo 
export LEGACYPIPE_DIR=$LEGACY_SURVEY_DIR/dr8code/legacypipe
export LEGACYZPTS_DIR=$LEGACY_SURVEY_DIR/dr8code/legacyzpts

# This file has most of the needed environment variables.
source $LEGACYPIPE_DIR/bin/legacypipe-env

# Define LEGACY_SURVEY_DIR *after* legacypipe-env is sourced.
export LEGACY_SURVEY_DIR=/global/project/projectdirs/cosmo/work/legacysurvey/dr8a

export PYTHONPATH=$QDO_DIR:$PYTHONPATH
export PYTHONPATH=$LEGACYPIPE_DIR/py:$PYTHONPATH
export PYTHONPATH=$LEGACYZPTS_DIR/py:$PYTHONPATH

export PATH=$QDO_DIR/bin:$PATH
export PATH=$LEGACYPIPE_DIR/bin:$PATH
export PATH=$LEGACYZPTS_DIR/bin:$PATH

export QDO_BACKEND=postgres
export QDO_BATCH_PROFILE=cori
export QDO_DB_HOST=nerscdb03.nersc.gov
export QDO_DB_NAME=desirun
export QDO_DB_PASS=ask_a_friend_for_this
export QDO_DB_USER=desirun_admin

# Some NERSC-specific options to get MPI working properly.
#  - Force MKL single-threaded and avoid problems with MPI and Python multiprocessing
#  - https://software.intel.com/en-us/articles/using-threaded-intel-mkl-in-multi-thread-application
export OMP_NUM_THREADS=1
export MKL_NUM_THREADS=1
export KMP_AFFINITY=disabled
export MPICH_GNI_FORK_MODE=FULLCOPY
```

### Launch the everything.

### Rsync everything to `project`.

```bash
cd $DW_PERSISTENT_SCRATCH/dr8a
rsync -auv zpts $LEGACY_SURVEY_DIR >> rsync-zpts.log 2>&1 &
rsync -auvR calib/*/*-merged $LEGACY_SURVEY_DIR >> rsync-calib.log 2>&1 &
```

### Build the merged *survey-ccds* and *annotated-ccds* files (and their KD-tree cousins).

```bash
./dr8-merge-zpts.sh
```
Might have to put this in a `slurm` script.  This also generates the KD-tree version of these files.

### Optional: update ccd_cuts and apply depth cuts.

### Run legacypipe

```bash
cd $DW_PERSISTENT_SCRATCH/dr8a/runbrick
rsync -auvR tractor* $LEGACY_SURVEY_DIR >> rsync-tractor.log 2>&1 &
rsync -auvR coadd $LEGACY_SURVEY_DIR >> rsync-coadd.log 2>&1 &
rsync -auvR metrics $LEGACY_SURVEY_DIR >> rsync-metrics.log 2>&1 &
```

```bash
qdo launch dr8a 256 --cores_per_worker 8 --walltime=00:30:00 \
  --script ./dr8-runbrick.sh --batchqueue debug --keep_env --batchopts "--bbf=bb.conf"
```


### Update the viewer

```bash
cd /global/project/projectdirs/cosmo/webapp/viewer-dev/
```
edit load-layer.py and run it!  Takes a long time.  Then “touch wsgi.py” and hit reload